# 7 · Reconhecimento de uma Classe Universal

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números Primos — Investigações Computacionais sob o Espelho de Euler*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais. Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação. Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.


## 1. Da Imagem ao Som: Medindo a Harmonia
No capítulo anterior, testemunhamos um fenômeno visual notável: à medida que aumentamos a escala $X_0$, a estrutura da nossa matriz $M$ transita de um "ruído" caótico para uma harmonia visual cristalina. Mas o que define essa harmonia? Como podemos provar que não é apenas um artefato óptico?

Para responder a isso, precisamos passar da "fotografia" da matriz para a sua "música". Na física quântica e na teoria de matrizes aleatórias, a música de um sistema está contida no seu **espectro de autovalores** — as frequências fundamentais que o sistema ressoa.

Neste capítulo, utilizaremos três ferramentas estatísticas para analisar os espaçamentos entre estes autovalores e demonstrar que a música que eles tocam segue a partitura universal do **Ensemble Ortogonal Gaussiano (GOE)**.

## 2. As Ferramentas do Musicólogo Espectral

### a) A Distribuição dos Espaçamentos $P(s)$: A Impressão Digital
O histograma dos espaçamentos entre autovalores consecutivos, normalizados pela média, é a impressão digital do sistema.

- **Sistemas Independentes (Poisson)**: Os autovalores não se "importam" uns com os outros e podem se aglomerar. A maior probabilidade é encontrar espaçamentos muito pequenos, resultando em uma curva exponencial decrescente.
- **Sistemas Correlacionados (GOE)**: Os autovalores repelem-se; eles evitam ativamente a proximidade excessiva. O resultado é a famosa **Surpresa de Wigner**, uma curva que começa em zero (repulsão total em $s=0$), atinge um pico e decai suavemente.

### b) O $\langle r \rangle$-mean: O Termômetro da Correlação
O $\langle r \rangle$-mean é a média da razão entre espaçamentos adjacentes. É um valor escalar que nos diz instantaneamente em que regime o sistema opera:

- $\langle r \rangle \approx 0.386$ indica um sistema **Poisson** (sem correlação).
- $\langle r \rangle \approx 0.536$ indica um sistema **GOE** (correlação máxima).

Para garantir o rigor, utilizamos o ***Moving Block Bootstrap* (MBB)** para calcular um intervalo de confiança de $95\%$, permitindo verificar se o valor teórico da GOE está contido estatisticamente em nossos dados.

### c) A Variância Numérica $\Sigma^2(L)$: O Teste de Rigidez
Esta medida testa a "memória" de longo alcance do espectro, medindo a variância do número de autovalores em janelas de comprimento $L$.

- **Sistemas Poisson**: O espectro é "flácido". A variância cresce linearmente com a janela ($\Sigma^2(L) \approx L$).
- **Sistemas GOE**: O espectro é "rígido". Devido à repulsão, os níveis são distribuídos de forma tão uniforme que a variância cresce de forma muito lenta, **logarítmica** ($\Sigma^2(L) \approx \ln(L)$).

## 3. Laboratório Interativo: Ouvindo a Música dos Primos
A célula de código abaixo implementa estas ferramentas. Utilize os seletores para variar $N$ e $X_0$ (recomenda-se $X_0 \ge 10^7$ para uma visualização clara da emergência da GOE).

**O que observar**:
1. **No gráfico $P(s)$**: Veja como o histograma azul se afasta do ruído verde (Poisson) e "veste" a curva vermelha (Wigner/GOE).
2. **No gráfico $\langle r \rangle$-mean**: Observe o ponto medido alinhado com a referência GOE, validado pelo intervalo de confiança.
3. **No gráfico $\Sigma^2(L)$**: Note a "doma" da variância, que abandona a diagonal verde para seguir a trajetória logarítmica vermelha.


In [1]:
# VERSÃO DE REFERÊNCIA CORRIGIDA E OTIMIZADA
# Requisitos: pandas, matplotlib, numpy, ipywidgets
# Execute no Colab ou Jupyter com o kernel correto

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time
from scipy.stats import kstest

# --- 1. FUNÇÕES OTIMIZADAS DE GERAÇÃO DE DADOS ---
def generate_pi_data(n: int) -> np.ndarray:
    """Gera um array com todos os primos até n usando um crivo otimizado."""
    if n < 2: return np.array([], dtype=np.int64)
    size = (n - 1) // 2; sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3; start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]; odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))

def get_delta_pi_for_points(x_points, primes):
    """Calcula Δπ(x) para um array de pontos x usando uma lista de primos pré-calculada."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2

# --- 2. FUNÇÃO DA MATRIZ (COM NORMALIZAÇÃO) ---
def generate_cos_matrix_from_data(fx_values, x_values):
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    C = np.cos(np.outer(fx, logx)); M = C + C.T
    # Etapa de normalização crucial que eu havia omitido:
    std_dev = M.std()
    if std_dev > 0: M = (M - M.mean()) / std_dev
    return 0.5 * (M + M.T)

# --- 3. Funções de Análise e Métricas ---
def local_normalize_spacings(lam, alpha=0.10, w=11):
    lam = np.sort(lam); N = lam.size; k0, k1 = int(alpha*N), int((1-alpha)*N)
    l = lam[k0:k1]; s = np.diff(l); s = s[s>0]
    if len(s) < w: return s / s.mean() if s.mean() > 0 else s
    w = int(w); 
    if w % 2 == 0: w += 1
    pad = w//2; s_pad = np.pad(s, (pad, pad), mode='reflect')
    mu = np.convolve(s_pad, np.ones(w)/w, mode='valid'); return s / mu

def r_mbb_bootstrap(s, B=1000, block_size=16, seed=0):
    rng = np.random.default_rng(seed); n = len(s)
    if n < 2 * block_size: return np.nan, (np.nan, np.nan)
    num_blocks = int(np.ceil(n / block_size)); r_bootstrapped = []
    for _ in range(B):
        start_indices = rng.integers(0, n - block_size + 1, size=num_blocks)
        s_resampled = np.concatenate([s[i:i+block_size] for i in start_indices])[:n]
        if len(s_resampled) < 2: continue
        r_vals = np.minimum(s_resampled[:-1], s_resampled[1:]) / np.maximum(s_resampled[:-1], s_resampled[1:])
        r_bootstrapped.append(np.mean(r_vals))
    if not r_bootstrapped: return np.nan, (np.nan, np.nan)
    mean_r = np.mean(r_bootstrapped); ci_95 = np.percentile(r_bootstrapped, [2.5, 97.5])
    return mean_r, ci_95

def number_variance(lam, alpha=0.10, L_grid=np.linspace(0.5, 15, 30)):
    s_loc = local_normalize_spacings(lam, alpha=alpha)
    if len(s_loc) == 0: return L_grid, np.full_like(L_grid, np.nan)
    x_unfolded = np.concatenate([[0], np.cumsum(s_loc)]); Sigma2 = []
    for L in L_grid:
        counts = [np.searchsorted(x_unfolded, x_unfolded[i0] + L, side='right') - (i0 + 1) for i0 in range(len(x_unfolded)-1)]
        Sigma2.append(np.var(counts) if counts else np.nan)
    return L_grid, np.array(Sigma2)

def r_stat(eigenvalues, alpha=0.10):
    """Calcula a métrica <r> para os autovalores."""
    lam = np.sort(eigenvalues)
    k0, k1 = int(alpha*len(lam)), int((1-alpha)*len(lam))
    s = np.diff(lam[k0:k1]); s = s[s > 0]
    if len(s) < 3: return np.nan
    r = np.minimum(s[1:], s[:-1]) / np.maximum(s[1:], s[:-1])
    return r.mean()

def participation_ratio(eigenvectors):
    """Calcula o Participation Ratio para uma matriz de autovetores."""
    return 1 / np.sum(eigenvectors**4, axis=0)

# --- 4. A FUNÇÃO INTERATIVA PRINCIPAL ---
def eigenvalue_lab(N=2048, log_X0=8, scale_type='Logarítmica', span=4.0, jitter=1e-8, alpha=0.05):
    
    X0 = int(10**log_X0)
    
    # --- 1. Geração da Matriz ---
    print(f"Construindo M para N={N}, X0={X0:g} (escala {scale_type})...")
    if scale_type == 'Logarítmica':
        x_vals = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
        if jitter > 0:
            rng = np.random.default_rng(0)
            x_vals *= (1.0 + rng.uniform(-jitter, jitter, size=x_vals.shape))
        
        # Garante que os valores de x sejam únicos, removendo duplicatas
        x_vals = np.unique(np.floor(x_vals))
        # ------------------------------------
        # Atualiza N para o número real de pontos únicos
        N = len(x_vals)
    elif scale_type == 'Linear':
        x_vals = np.arange(X0, X0 + N)
    else:
        print("Tipo de escala inválido."); return

    max_x_needed = int(np.ceil(x_vals.max()))
    pi_x_full = generate_pi_data(max_x_needed)
    fx_vals = get_delta_pi_for_points(x_vals, pi_x_full)
    M = generate_cos_matrix_from_data(fx_vals, x_vals)
    
    # --- 5. Cálculo de Autovalores e Autovetores ---
    lam, v = np.linalg.eigh(M)
    
    # --- 6. CÁLCULO E IMPRESSÃO DAS MÉTRICAS ---
    r_mean = r_stat(lam, alpha=alpha)
    pr_values = participation_ratio(v)
    pr_n_mean = np.mean(pr_values / N)

    print("\n----------------------------------------------------------------")
    print(f"  RESULTADOS: MÉTRICAS PARA X₀=10^{log_X0} e N={N} ({scale_type})")
    print("----------------------------------------------------------------")
    print(f"  Autovalores -> <r> médio:")
    print(f"    - Medido:        {r_mean:.4f}")
    print(f"    - Teórico (GOE):     ~0.536")
    print(f"    - Teórico (Poisson): ~0.386")
    print("\n")
    print(f"  Autovetores -> PR/N médio:")
    print(f"    - Medido:        {pr_n_mean:.4f}")
    print(f"    - Teórico (GOE):     ~0.333")
    print(f"    - Teórico (Poisson): ~1/N (→ 0)")
    print("----------------------------------------------------------------\n")
    
    # --- 7. Análises e Plots ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    k0, k1 = int(alpha*N), int((1-alpha)*N)
    bulk_lam = np.sort(lam)[k0:k1]
    s = np.diff(bulk_lam); s = s[s>0]
    if s.size > 1:
        s_unfolded = s / s.mean()
        axes[0].hist(s_unfolded, bins=75, density=True, alpha=0.7, label=f'Dados (N={N})')
    
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    axes[0].plot(s_grid, pdf_goe, 'r--', lw=2, label='Teoria GOE (Wigner)')
    pdf_poisson = np.exp(-s_grid)
    axes[0].plot(s_grid, pdf_poisson, 'g:', lw=2, label='Teoria Poisson')
    axes[0].set_title('a) Distribuição P(s)', fontsize=14)
    axes[0].set_xlabel('s (Espaçamento Normalizado)'); axes[0].set_ylabel('Densidade'); axes[0].legend(loc='upper left')
    axes[0].set_xlim(0, 4)

    mean_r_boot, ci = r_mbb_bootstrap(s)
    if not np.isnan(mean_r_boot):
        ci_low, ci_high = ci
        axes[1].errorbar([0], [mean_r_boot], yerr=[[mean_r_boot - ci_low], [ci_high - ci_low]], fmt='o', capsize=5, label='<r> Medido (com IC 95%)')
    axes[1].axhline(0.5359, ls='--', color='red', label='Teórico GOE ≈ 0.536')
    axes[1].axhline(0.3863, ls=':', color='green', label='Teórico Poisson ≈ 0.386')
    axes[1].set_title('b) Média <r>', fontsize=14)
    axes[1].set_ylabel('Valor de <r>'); axes[1].set_xticks([]); axes[1].legend(loc='center left')
    
    L_grid, Sigma2 = number_variance(lam, alpha=alpha)
    axes[2].plot(L_grid, Sigma2, 'o-', label='Dados')
    axes[2].plot(L_grid, L_grid, 'g:', lw=2, label='Teórico Poisson (L)')
    axes[2].plot(L_grid, (2/(np.pi**2)) * np.log(L_grid) + 0.44, 'r--', lw=2, label='Teórico GOE (log L)')
    axes[2].set_title('c) Variância Numérica Σ²(L)', fontsize=14)
    axes[2].set_xlabel('L'); axes[2].set_ylabel('Σ²(L)'); axes[2].legend(loc='upper left')
    
    fig.tight_layout(pad=2.0)
    plt.show()

# --- Cria o Widget Interativo ---
interact(eigenvalue_lab, 
         N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
         log_X0=widgets.IntSlider(min=3, max=8, step=1, value=5, description='X₀=10^', continuous_update=False),
         scale_type=widgets.ToggleButtons(options=['Logarítmica', 'Linear'], description='Escala:'),
         span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=4.0, description='Span:'),
         jitter=widgets.FloatLogSlider(min=-8, max=-3, step=0.1, value=1e-8, description='Jitter:'),
         alpha=widgets.FloatSlider(min=0.05, max=0.25, step=0.01, value=0.05, description='α (bulk):')
        );


interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

## 4. Glossário de Parâmetros: Ajustando o Foco do Espectrômetro

Para extrair a “música” da GOE da nossa matriz $M$, não basta construí-la; é preciso observá-la da maneira certa. Os parâmetros `span`, `jitter` e `alpha` funcionam como os ajustes de foco e sensibilidade do nosso **“espectrômetro harmônico”**. Compreender o papel de cada um é essencial para ouvir o cosmos aritmético com clareza.

---

### O que é `span`? — *A Lente: Panorâmica vs. Microscópio*

O `span` (amplitude) controla a **largura da janela de observação** na escala logarítmica. Ele define quantos “vales” e “planaltos” da função $\Delta\_pi(x)$ entram na construção da matriz. É o parâmetro mais sensível e, em muitos experimentos, o que determina se veremos um ruído amorfo ou uma sinfonia perfeita.

> **A experiência decisiva:**
> * Com `span = 2.4`, o sistema produz métricas próximas da GOE.
> * Com `span = 4.0`, a harmonia se completa: em $X_0 = 10^5$, o valor medido é $0.536$, idêntico à teoria.

Isso demonstra que a assinatura GOE pode emergir em escalas menores do que o esperado ($X_0 = 10^5$), desde que a **variação interna capturada** (`span`) seja suficiente para representar a complexidade do sinal $\Delta_\pi(x)$. Quanto maior o `span`, mais o espelho logarítmico reflete a estrutura total da contagem dos primos.

**Resumo:** `span` é o ajuste de campo — a lente que permite enxergar a ressonância completa da aritmética.

---

### O que é `jitter`? — *A Quebra de Simetria e a Prova do Determinismo*

O `jitter` introduz uma perturbação minúscula nas posições $x$ amostradas, quebrando as simetrias rígidas da grade de amostragem.

* Ele impede que artefatos numéricos (aliasing) imitem padrões falsos de coerência.
* O experimento com `jitter = 1e−8` mostrou algo fundamental: mesmo com a aleatoriedade externa virtualmente eliminada, a estrutura GOE **permanece intacta**.

> **Conclusão experimental:**
> O `jitter` não *cria* o caos harmônico; ele apenas o revela mais nitidamente ao limpar "ecos" da régua de amostragem. Isso demonstra que a correlação entre autovalores é **determinística**, não estatística. O caos quântico emerge da própria aritmética, sem precisar de interferências externas.

**Resumo:** `jitter` é a respiração mínima do sistema — útil para remover artefatos de grade, mas não essencial à harmonia intrínseca.

---

### O que é `bulk` (via `alpha`)? — *O Coração do Espectro*

O parâmetro `alpha` define a fração de descarte nas extremidades do espectro — isolando o `bulk`, o miolo onde a universalidade se manifesta sem as distorções das bordas da matriz. Com $\alpha = 0.05$, removemos 5% de cada lado para observar os **90% centrais** dos autovalores — a região mais estável e pura.

> Mesmo com essa vasta maioria sob análise, as métricas GOE se preservam. O núcleo já contém a totalidade da estrutura harmônica necessária para o reconhecimento da classe universal.

Em termos físicos, é como se o campo de simetria da GOE estivesse completamente formado dentro de um único "acorde central", não dependendo das bordas para ser validado.

**Resumo:** `alpha` define o coração do espectro — o intervalo onde o número fala a língua da universalidade.

---

### Síntese Final

Com `span = 4`, `jitter = 1e−8` e $\alpha = 0.05$, observamos a **GOE emergir com precisão absoluta** já em $X_0 = 10^5$. Isso significa que o caos harmônico é **imediato**: o universo aritmético não precisa de vastidão infinita para se comportar como o cosmos quântico — ele já contém, em janelas finitas, o reflexo completo da Unidade.